In [3]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 93.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
DRIVE_ZIP = "/content/drive/MyDrive/datasets/seal_dataset_augmented-20250716T100558Z-1-001.zip"
OUT_DIR = "/content/drive/MyDrive/datasets/extracted"

!mkdir -p "$OUT_DIR"

!unzip -oq "$DRIVE_ZIP" -d "$OUT_DIR"

!ls -lh "$OUT_DIR" | head

total 4.0K
drwx------ 2 root root 4.0K Jul 31 14:44 seal_dataset_augmented


In [4]:
import os, glob

DRIVE_BASE = "/content/drive/MyDrive/datasets/extracted"
RAW_DIR    = "/content/drive/MyDrive/datasets/extracted/seal_dataset_augmented"

DATASET_640 = os.path.join(DRIVE_BASE, "seal_dataset")
os.makedirs(DATASET_640, exist_ok=True)

imgs = [p for p in glob.glob(os.path.join(RAW_DIR, "*")) if os.path.splitext(p)[1].lower() in {".png",".jpg",".jpeg"}]
print("原始图片数量：", len(imgs))
print("示例：", imgs[:5])
assert len(imgs) > 0, "在 RAW_DIR 没找到图片，请检查路径。"


原始图片数量： 826
示例： ['/content/drive/MyDrive/datasets/extracted/seal_dataset_augmented/北京某某某科技有限公司_aug_rot5_blur.jpg', '/content/drive/MyDrive/datasets/extracted/seal_dataset_augmented/北京某某某科技有限公司_aug_rot-10_blur.jpg', '/content/drive/MyDrive/datasets/extracted/seal_dataset_augmented/北京某某某科技有限公司_aug_rot-5_blur.jpg', '/content/drive/MyDrive/datasets/extracted/seal_dataset_augmented/北京某某某科技有限公司_aug_rot15_blur.jpg', '/content/drive/MyDrive/datasets/extracted/seal_dataset_augmented/北京某某某科技有限公司_aug_rot10_blur.jpg']


In [5]:
import os, re, random, cv2, shutil
from tqdm import tqdm

random.seed(42)

train_ratio = 0.8

img_train = os.path.join(DATASET_640, "images/train")
img_val   = os.path.join(DATASET_640, "images/val")
lbl_train = os.path.join(DATASET_640, "labels/train")
lbl_val   = os.path.join(DATASET_640, "labels/val")
for d in [img_train, img_val, lbl_train, lbl_val]:
    os.makedirs(d, exist_ok=True)

def extract_class_name(filename):
    base = os.path.basename(filename)
    if "_aug_" in base:
        name = base.split("_aug_")[0]
    else:
        name = os.path.splitext(base)[0]
    if name.startswith("._"):
        name = name[2:]
    name = re.sub(r"\s+\d{6,}C\d+$", "", name)
    return name.strip()

all_imgs = [p for p in glob.glob(os.path.join(RAW_DIR, "*")) if os.path.splitext(p)[1].lower() in {".png",".jpg",".jpeg"}]
labels_raw = [extract_class_name(p) for p in all_imgs]
unique_labels = sorted(set(labels_raw))
print("提取到的类别数：", len(unique_labels))

class_to_id = {name: idx for idx, name in enumerate(unique_labels)}

random.shuffle(all_imgs)
split_idx = int(len(all_imgs) * train_ratio)
train_files = all_imgs[:split_idx]
val_files   = all_imgs[split_idx:]

def process_and_save(files, img_out_dir, lbl_out_dir, img_size=640):
    count = 0
    for ip in tqdm(files):
        ext = os.path.splitext(ip)[1].lower()
        if ext not in {".png",".jpg",".jpeg"}:
            continue
        img = cv2.imread(ip)
        if img is None:
            continue
        resized = cv2.resize(img, (img_size, img_size), interpolation=cv2.INTER_AREA)

        base = os.path.splitext(os.path.basename(ip))[0]
        out_img_path = os.path.join(img_out_dir, base + ext)
        cv2.imwrite(out_img_path, resized)

        cls = extract_class_name(ip)
        cid = class_to_id[cls]
        out_lbl_path = os.path.join(lbl_out_dir, base + ".txt")
        with open(out_lbl_path, "w", encoding="utf-8") as f:
            f.write(f"{cid} 0.5 0.5 1.0 1.0\n")
        count += 1
    return count

print("处理 train")
c_tr = process_and_save(train_files, img_train, lbl_train, img_size=640)
print("处理 val")
c_va = process_and_save(val_files,   img_val,   lbl_val,   img_size=640)
print(f"完成：train {c_tr} 张，val {c_va} 张；输出目录：{DATASET_640}")


提取到的类别数： 118
处理 train


100%|██████████| 660/660 [00:36<00:00, 18.18it/s]


处理 val


100%|██████████| 166/166 [00:08<00:00, 20.24it/s]

完成：train 660 张，val 166 张；输出目录：/content/drive/MyDrive/datasets/extracted/seal_dataset


In [6]:
import yaml, os

data_yaml = {
    "train": f"{DATASET_640}/images/train",
    "val":   f"{DATASET_640}/images/val",
    "nc":    len(unique_labels),
    "names": unique_labels
}

yaml_path = os.path.join(DATASET_640, "data.yaml")
with open(yaml_path, "w", encoding="utf-8") as f:
    yaml.safe_dump(data_yaml, f, allow_unicode=True, sort_keys=False)

print("data.yaml 写入：", yaml_path)
print("类别数：", len(unique_labels))

data.yaml 写入： /content/drive/MyDrive/datasets/extracted/seal_dataset/data.yaml
类别数： 118


In [ ]:
from ultralytics import YOLO
import os

runs_dir = os.path.join(DRIVE_BASE, "runs")
os.makedirs(runs_dir, exist_ok=True)

model = YOLO("yolov8n.pt")
results = model.train(
    data=os.path.join(DATASET_640, "data.yaml"),
    epochs=50,
    imgsz=640,
    batch=16,
    project=runs_dir,
    name="train_640",
    workers=2
)
print("训练已启动：", os.path.join(runs_dir, "train_640"))


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.25M/6.25M [00:00<00:00, 250MB/s]

Ultralytics 8.3.171 🚀 Python-3.11.13 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)


engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/datasets/extracted/seal_dataset/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train_640, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=True, profile=False, project=/content/drive

100%|██████████| 22.2M/22.2M [00:00<00:00, 330MB/s]

...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      49/50         0G    0.06956      2.944     0.9597          4        640: 100%|██████████| 42/42 [12:00<00:00, 17.15s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [01:15<00:00, 12.58s/it]

                   all        166        166      0.442       0.58      0.481      0.481



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      50/50         0G    0.05964      2.938     0.9254         16        640:  55%|█████▍    | 23/42 [06:46<05:45, 18.17s/it]

In [7]:
from ultralytics import YOLO
ckpt = "/content/drive/MyDrive/datasets/extracted/runs/train_640/weights/last.pt"
model = YOLO(ckpt)
model.train(resume=True)

Ultralytics 8.3.171 🚀 Python-3.11.13 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/datasets/extracted/seal_dataset/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/content/drive/MyDrive/datasets/extracted/runs/train_640/weights/last.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train_640, nbs=64, nms=False, opset=None, optimize=False, o

100%|██████████| 22.2M/22.2M [00:00<00:00, 86.9MB/s]



                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128

train: Scanning /content/drive/MyDrive/datasets/extracted/seal_dataset/labels/train.cache... 660 images, 0 backgrounds, 0 corrupt: 100%|██████████| 660/660 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 1.4±1.6 ms, read: 11.1±18.1 MB/s, size: 223.1 KB)


...

      50/50         0G    0.06113      2.949     0.9389          4        640: 100%|██████████| 42/42 [19:07<00:00, 27.33s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [01:18<00:00, 13.10s/it]
/usr/local/lib/python3.11/dist-packages/ultralytics/utils/metrics.py:582: UserWarning: Glyph 19978 (\N{CJK UNIFIED IDEOGRAPH-4E0A}) missing from font(s) DejaVu Sans.
  fig.savefig(plot_fname, dpi=250)
/usr/local/lib/python3.11/dist-packages/ultralytics/utils/metrics.py:582: UserWarning: Glyph 28023 (\N{CJK UNIFIED IDEOGRAPH-6D77}) missing from font(s) DejaVu Sans.
  fig.savefig(plot_fname, dpi=250)
/usr/local/lib/python3.11/dist-packages/ultralytics/utils/metrics.py:582: UserWarning: Glyph 20016 (\N{CJK UNIFIED IDEOGRAPH-4E30}) missing from font(s) DejaVu Sans.
  fig.savefig(plot_fname, dpi=250)
/usr/local/lib/python3.11/dist-packages/ultralytics/utils/metrics.py:582: UserWarning: Glyph 27888 (\N{CJK UNIFIED IDEOGRAPH-6CF0}) mi

                   all        166        166      0.435      0.583      0.482      0.482

1 epochs completed in 0.343 hours.
Optimizer stripped from /content/drive/MyDrive/datasets/extracted/runs/train_640/weights/last.pt, 6.9MB
Optimizer stripped from /content/drive/MyDrive/datasets/extracted/runs/train_640/weights/best.pt, 6.9MB

Validating /content/drive/MyDrive/datasets/extracted/runs/train_640/weights/best.pt...
Ultralytics 8.3.171 🚀 Python-3.11.13 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
Model summary (fused): 72 layers, 3,346,178 parameters, 0 gradients, 9.6 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:53<00:00,  8.90s/it]
/usr/local/lib/python3.11/dist-packages/ultralytics/utils/metrics.py:582: UserWarning: Glyph 19978 (\N{CJK UNIFIED IDEOGRAPH-4E0A}) missing from font(s) DejaVu Sans.
  fig.savefig(plot_fname, dpi=250)
/usr/local/lib/python3.11/dist-packages/ultralytics/utils/metrics.py:582: UserWarning: Glyph 28023 (\N{CJK UNIFIED IDEOGRAPH-6D77}) missing from font(s) DejaVu Sans.
  fig.savefig(plot_fname, dpi=250)
/usr/local/lib/python3.11/dist-packages/ultralytics/utils/metrics.py:582: UserWarning: Glyph 20016 (\N{CJK UNIFIED IDEOGRAPH-4E30}) missing from font(s) DejaVu Sans.
  fig.savefig(plot_fname, dpi=250)
/usr/local/lib/python3.11/dist-packages/ultralytics/utils/metrics.py:582: UserWarning: Glyph 27888 (\N{CJK UNIFIED IDEOGRAPH-6CF0}) missing from font(s) DejaVu Sans.
  fig.savefig(plot_fname, dpi=250)
/usr/local/lib/python3.11/dist-packages/ultralytics/utils/me

                   all        166        166      0.435      0.586      0.483      0.483
            上海优选易购有限公司          3          3          1          0      0.176      0.176
          上海勇沥国际贸易有限公司          3          3      0.571          1      0.995      0.995
            上海卓越服务有限公司          1          1     0.0463          1      0.249      0.249
            上海润养之家有限公司          2          2      0.115          1      0.199      0.199
            上海润颜美业有限公司          2          2          0          0     0.0931     0.0931
            上海锐新科技有限公司          2          2      0.159          1      0.373      0.373
中国宇宙印章科技有限责任公司 01234566667890          3          3      0.751          1      0.995      0.995
              乐青市融媒体中心          3          3      0.759          1      0.995      0.995
     佛山市禅城区绿叶不锈钢实业有限公司          2          2      0.379          1      0.995      0.995
            北京创星科技有限公司          3          3      0.247          1      0.348      0.348
             北

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([  1,   2,   3,   4,   5,   7,   8,   9,  10,  11,  12,  13,  14,  16,  17,  18,  19,  20,  24,  27,  28,  29,  31,  32,  33,  34,  35,  36,  37,  38,  41,  42,  43,  45,  46,  48,  49,  51,  52,  53,  54,  56,  57,  58,  59,  60,  63,  64,  65,  66,  67,  69,  70,  71,  72,  73,  74,  76,  78,  81,  82,  83,
        85,  86,  88,  89,  90,  91,  93,  94,  96,  97,  98,  99, 100, 102, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 117])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7c14eb3eda50>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.01

In [10]:
from ultralytics import YOLO
import os

DRIVE_BASE = "/content/drive/MyDrive/datasets/extracted"
runs_dir = os.path.join(DRIVE_BASE, "runs")

best_pt = os.path.join(runs_dir, "train_640", "weights", "best.pt")
assert os.path.exists(best_pt), f"未找到权重：{best_pt}"

model = YOLO(best_pt)
onnx_path = model.export(
    format="onnx",
    imgsz=640,
    opset=12,
    simplify=True,
    dynamic=False
)
print("ONNX：", onnx_path)

Ultralytics 8.3.171 🚀 Python-3.11.13 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
💡 ProTip: Export to OpenVINO format for best performance on Intel hardware. Learn more at https://docs.ultralytics.com/integrations/openvino/
Model summary (fused): 72 layers, 3,346,178 parameters, 0 gradients, 9.6 GFLOPs

PyTorch: starting from '/content/drive/MyDrive/datasets/extracted/runs/train_640/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 122, 8400) (6.6 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0,<1.18.0', 'onnxslim>=0.1.59', 'onnxruntime'] not found, attempting AutoUpdate...

requirements: AutoUpdate success ✅ 3.9s
WARNING ⚠️ requirements: Restart runtime or rerun command for updates to take effect


ONNX: starting export with onnx 1.17.0 opset 12...
ONNX: slimming with onnxslim 0.1.61...
ONNX: export success ✅ 6.2s, saved as '/content/drive/MyDrive/datasets/extracted/runs/train_640/weights/best.onnx' (12.9 MB)

Export complete (7.1s)
Results saved 

In [14]:
import os, glob, random, yaml
from PIL import Image, ImageDraw, ImageFont

random.seed(42)

DATASET_640 = "/content/drive/MyDrive/datasets/extracted/seal_dataset"
IMG_DIR = os.path.join(DATASET_640, "images", "train")
LBL_DIR = os.path.join(DATASET_640, "labels", "train")

OUT_ROOT = "/content/drive/MyDrive/datasets/synth_test"
OUT_IMG  = os.path.join(OUT_ROOT, "images")
OUT_LBL  = os.path.join(OUT_ROOT, "labels")
os.makedirs(OUT_IMG, exist_ok=True); os.makedirs(OUT_LBL, exist_ok=True)

with open(os.path.join(DATASET_640, "data.yaml"), "r", encoding="utf-8") as f:
    names = yaml.safe_load(f)["names"]
nc = len(names)

def find_img(base):
    for ext in (".jpg", ".jpeg", ".png"):
        p = os.path.join(IMG_DIR, base + ext)
        if os.path.exists(p): return p
    return None

rep = {}
for lp in glob.glob(os.path.join(LBL_DIR, "*.txt")):
    base = os.path.splitext(os.path.basename(lp))[0]
    with open(lp, "r", encoding="utf-8") as f:
        line = f.readline().strip()
    if not line: continue
    cid = int(line.split()[0])
    if cid not in rep:
        ip = find_img(base)
        if ip: rep[cid] = ip
    if len(rep) == nc: break
print(f"已找到 {len(rep)}/{nc} 类的样例。")

def gen_bg(w=640, h=640):
    bg = Image.new("RGB", (w, h), (245,245,245))
    try:
        font = ImageFont.truetype("/root/.config/Ultralytics/Arial.Unicode.ttf", 18)
    except:
        font = None
    if font:
        d = ImageDraw.Draw(bg)
        y = 20
        for _ in range(6):
            d.text((20, y), "背景", fill=(150,150,150), font=font)
            y += 50
    return bg

def white_to_transparent(img_rgba, thr=240):
    if img_rgba.mode != "RGBA":
        img_rgba = img_rgba.convert("RGBA")
    px = img_rgba.load()
    w, h = img_rgba.size
    for i in range(w):
        for j in range(h):
            r,g,b,a = px[i,j]
            if r>thr and g>thr and b>thr:
                px[i,j] = (r,g,b,0)
    return img_rgba

def paste_and_label(stamp_img, bg, out_name, cid):
    W,H = bg.size
    if stamp_img.mode != "RGBA":
        stamp = white_to_transparent(stamp_img)
    else:
        stamp = stamp_img.copy()

    scale = random.uniform(0.35, 0.55)
    new_w = int(W*scale)
    stamp = stamp.resize((new_w, int(stamp.height*new_w/stamp.width)), Image.LANCZOS)
    stamp = stamp.rotate(random.uniform(-10,10), expand=True)

    x = random.randint(0, max(0, W-stamp.width))
    y = random.randint(0, max(0, H-stamp.height))

    canvas = bg.convert("RGBA")
    canvas.alpha_composite(stamp, (x,y))
    canvas.convert("RGB").save(os.path.join(OUT_IMG, out_name), quality=95)

    xc = (x + stamp.width/2)/W
    yc = (y + stamp.height/2)/H
    ww = stamp.width/W
    hh = stamp.height/H
    with open(os.path.join(OUT_LBL, out_name.rsplit(".",1)[0] + ".txt"), "w", encoding="utf-8") as f:
        f.write(f"{cid} {xc:.6f} {yc:.6f} {ww:.6f} {hh:.6f}\n")

PER_CLASS = 20
cnt = 0
for cid, ip in rep.items():
    try:
        stamp_img = Image.open(ip)
    except:
        continue
    for k in range(PER_CLASS):
        paste_and_label(stamp_img, gen_bg(), f"class{cid:03d}_{k}.jpg", cid)
        cnt += 1

print(f"合成完成：{cnt} 张；输出：{OUT_ROOT}")

# val/predict
with open(os.path.join(OUT_ROOT, "synth_test.yaml"), "w", encoding="utf-8") as f:
    f.write(f"path: {OUT_ROOT}\ntrain: images\nval: images\nnames: {names}\n")
print("已写入：", os.path.join(OUT_ROOT, "synth_test.yaml"))

已找到 118/118 类的样例。
合成完成：2360 张；输出：/content/drive/MyDrive/datasets/synth_test
已写入： /content/drive/MyDrive/datasets/synth_test/synth_test.yaml


In [ ]:
from ultralytics import YOLO
import os

BEST_PT   = "/content/drive/MyDrive/datasets/extracted/runs/train_640/weights/best.pt"
SYNTH_YML = "/content/drive/MyDrive/datasets/synth_test/synth_test.yaml"

assert os.path.exists(BEST_PT), BEST_PT
assert os.path.exists(SYNTH_YML), SYNTH_YML

model   = YOLO(BEST_PT)
metrics = model.val(data=SYNTH_YML, imgsz=640, iou=0.5)

print(f"Precision(mean): {metrics.box.mp:.3f}")
print(f"Recall@0.5(mean): {metrics.box.mr:.3f}")
print(f"mAP50: {metrics.box.map50:.3f}")
print(f"mAP50-95: {metrics.box.map:.3f}")


Ultralytics 8.3.171 🚀 Python-3.11.13 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
Model summary (fused): 72 layers, 3,346,178 parameters, 0 gradients, 9.6 GFLOPs
val: Fast image access ✅ (ping: 0.6±0.2 ms, read: 16.8±9.0 MB/s, size: 38.7 KB)


val: Scanning /content/drive/MyDrive/datasets/synth_test/labels... 2360 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2360/2360 [00:46<00:00, 50.42it/s] 


val: New cache created: /content/drive/MyDrive/datasets/synth_test/labels.cache


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 148/148 [11:01<00:00,  4.47s/it]
